#Работа с файлами, pandas + парсинг
Собираем корпус эко-новостей с сайта новостного издания "Ведомости"

[По этой ссылке можно найти главную страницу](https://www.vedomosti.ru/ecology)

Что нам предстоит сделать:

достать все заголовки новостей в главной страницы + текст каждой новости

сохранить в датафрейм с колонками "ссылка", "дата", "заголовок", "текст"

сохранить датафрейм в файл

положить код и получившийся файл в свой репозиторий на гитхабе

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.vedomosti.ru/ecology'

In [3]:
page = rq.get(url) 
print(page)

<Response [200]>


In [4]:
soup = BeautifulSoup(page.text, features="html.parser")
print(soup.prettify())

<!DOCTYPE doctype html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" data-n-head-ssr="" lang="ru">
 <head>
  <title>
   «Ведомости.Экология» — приложение к газете «Ведомости»
  </title>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=no, minimum-scale=1, maximum-scale=1" data-n-head="ssr" name="viewport"/>
  <meta content="#fff6f1" data-n-head="ssr" name="theme-color"/>
  <meta content="max-image-preview:large" data-n-head="ssr" name="robots"/>
  <meta content="/browserconfig.xml" data-n-head="ssr" name="msapplication-config"/>
  <meta content="5UKfplA2CDg7gtsOuVFb6QPtKINUJzM-11wPEzqubM8" data-n-head="ssr" name="google-site-verification"/>
  <meta content="Ведомости.Экология" data-hid="apple-mobile-web-app-title" data-n-head="ssr" name="apple-mobile-web-app-title"/>
  <meta content="summary_large_image" data-hid="twitter:card" data-n-head="ssr" name="twitter:card"/>
  <meta content="@vedomosti" data-hid="t

In [5]:
urls = []
for link in soup.find_all('a'):
    urls.append(link.get('href'))
print(urls)

['https://www.vedomosti.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://vedomostisport.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://vedomosti-spb.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://www.vedomosti.ru/gorod?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://www.vedomosti.ru/partner?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://regatta.vedomosti.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://kp.vedomosti.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://www.vedomosti.ru/companies?utm_source=main&utm_campaign=desktop_main', 'https://events.vedomosti.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&utm_campaign=desktop_main', 'https://re.vedomosti.ru/?utm_source=vedomosti.ru/ecology&utm_medium=main&ut

In [6]:
releases = []
for link in soup.find_all('a'):
  if 'release' in link.get('href') and link.get('href') not in releases:
    releases.append(link.get('href'))
    print(releases)

['/ecology/release/2022/12/12']
['/ecology/release/2022/12/12', '/ecology/release/2022/12/08']
['/ecology/release/2022/12/12', '/ecology/release/2022/12/08', '/ecology/release/2022/11/10']
['/ecology/release/2022/12/12', '/ecology/release/2022/12/08', '/ecology/release/2022/11/10', '/ecology/release/2022/09/08']


In [7]:
realeases_links = []

for link in (releases): 
  realeases_links.append('https://www.vedomosti.ru' + link)

for l in realeases_links:
  print(l)

https://www.vedomosti.ru/ecology/release/2022/12/12
https://www.vedomosti.ru/ecology/release/2022/12/08
https://www.vedomosti.ru/ecology/release/2022/11/10
https://www.vedomosti.ru/ecology/release/2022/09/08


In [8]:
release_pages = []
for i in realeases_links:
  release_page = rq.get(i)
  release_soup = BeautifulSoup(release_page.text, features='html.parser')
  release_pages.append(release_soup)
  print(release_soup.prettify())

Выходные данные были обрезаны до нескольких последних строк (5000).
                 Развернуть обратно
                </title>
                <path _fill="#000" d="M9.03 3.53L7.97 2.47 3.94 6.5l4.03 4.03 1.06-1.06-2.22-2.22h4.476c2.148 0 3.964 1.86 3.964 4.25 0 2.39-1.816 4.25-3.964 4.25H6.222v1.5h5.064c3.059 0 5.464-2.617 5.464-5.75s-2.405-5.75-5.464-5.75H6.81l2.22-2.22z" pid="0">
                </path>
               </svg>
              </div>
             </div>
            </div>
            <div class="grid-cell__body">
             <a class="card-background articles-cards-list__card cols-2 rows-3 --background" data-vr-contentbox-url="/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii" data-vr-headline="Метан – недолговечный, но коварный" data-vr-title="Метан – недолговечный, но коварный" href="/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii">
              <div class="card-background__content">
               <div class=

In [9]:
article_urls = []
for soup in release_pages:
  for link in soup.find_all('a', {"class" : "articles-cards-list__card card-article cols-1 rows-3 --article"}):
    article_urls.append(link.get('href'))

  for link in soup.find_all('a', {"class" : "card-background articles-cards-list__card cols-2 rows-3 --background"}):
    article_urls.append(link.get('href'))
    print(article_urls)


['/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-i-ustoichivoe-razvitie', '/ecology/science_and_technology/articles/2022/12/12/954778-dobivayuschei-otrasli-prognoziruyut-diversifikatsiyu', '/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii']
['/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-i-ustoichivoe-razvitie', '/ecology/science_and_technology/articles/2022/12/12/954778-dobivayuschei-otrasli-prognoziruyut-diversifikatsiyu', '/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii', '/ecology/science_and_technology/articles/2022/12/08/954309-rossiya-stanet-samim-moschnim-ledokolom-v-mire', '/ecology/science_and_technology/articles/2022/12/08/954298-alternativa-bolshomu-atomu', '/ecology/protection_nature/articles/2022/12/08/954286-atomnaya-ekologiya', '/ecology/climate/articles/2022/12/08/954315-energetika-nulevim-sledom']
['/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-

In [10]:
full_article_links = []

for link in (article_urls): 
  full_article_links.append('https://www.vedomosti.ru' + link)

for l in full_article_links:
  print(l)

https://www.vedomosti.ru/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-i-ustoichivoe-razvitie
https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/12/954778-dobivayuschei-otrasli-prognoziruyut-diversifikatsiyu
https://www.vedomosti.ru/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii
https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/08/954309-rossiya-stanet-samim-moschnim-ledokolom-v-mire
https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/08/954298-alternativa-bolshomu-atomu
https://www.vedomosti.ru/ecology/protection_nature/articles/2022/12/08/954286-atomnaya-ekologiya
https://www.vedomosti.ru/ecology/climate/articles/2022/12/08/954315-energetika-nulevim-sledom
https://www.vedomosti.ru/ecology/regulation/articles/2022/11/10/949661-zapretyat-li-sobirat-kedrovie-orehi
https://www.vedomosti.ru/ecology/protection_nature/articles/2022/11/10/949663-v-rossii-otsutstvuet-spetsializ

In [11]:
article_pages = []
for i in full_article_links:
  article_page = rq.get(i)
  article_soup = BeautifulSoup(article_page.text, features='html.parser')
  print(article_soup.prettify())
  article_pages.append(article_soup)

Выходные данные были обрезаны до нескольких последних строк (5000).
          <!-- -->
          <!-- -->
          <!-- -->
         </p>
         <p class="the-footer__copyright-text">
          Рекламно-информационное приложение к газете «Ведомости». Зарегистрировано Федеральной службой по надзору в сфере связи, информационных технологий и массовых коммуникаций (Роскомнадзор) за номером ПИ № ФС 77 – 77720 от 17 января 2020 г.
          <!-- -->
          <!-- -->
          <!-- -->
         </p>
         <p class="the-footer__copyright-text">
          Любое использование материалов допускается только при соблюдении
          <a class="the-footer__copyright-link" href="/info/reprint_rules" target="_blank">
           правил перепечатки
          </a>
          <span>
           и при наличии гиперссылки на vedomosti.ru
          </span>
          <!-- -->
         </p>
         <p class="the-footer__copyright-text">
          Новости, аналитика, прогнозы и другие материалы, представ

In [12]:
def GetNews(url):
  page = rq.get(url)
  soup = BeautifulSoup(page.text, features="html.parser")
  title = soup.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
  
  time = soup.find('time').attrs['datetime']
  date = time[:10]

  text_list = soup.find_all('p', {'class' : 'box-paragraph__text'})
  text = []
  for i in text_list:
    text.append(i.text)
  final_text = ' '.join(text)
  
  return url, date, title, final_text

In [13]:
news = []
for i in full_article_links:
  news.append(GetNews(i))
print(news)

[('https://www.vedomosti.ru/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-i-ustoichivoe-razvitie', '2022-12-12', 'Климатические изменения и устойчивое развитие', 'Современные климатические изменения представляют глобальную проблему, для решения которой требуется готовность ведущих стран мира к взаимным уступкам и созданию благоприятных политических условий для согласованной совместной работы В настоящее время проблема климатических изменений находится в сфере пристального внимания не только ученых-экологов, но и широкой общественности — политиков, журналистов и даже рядовых граждан. Борьба с глобальным потеплением и сопряженный с этим спектр серьезных климатических вопросов постоянно присутствуют в повестке дня всех мировых саммитов, деловых форумов и являются неотъемлемой частью корпоративной повестки большинства крупных компаний. Современное климатологическое сообщество практически единодушно придерживается точки зрения, согласно которой в настоящее время климат

In [14]:
df = pd.DataFrame(news)
df.head()

,0,1,2,3
0,https://www.vedomosti.ru/ecology/climate/colum...,2022-12-12,Климатические изменения и устойчивое развитие,Современные климатические изменения представля...
1,https://www.vedomosti.ru/ecology/science_and_t...,2022-12-12,Добывающей отрасли прогнозируют диверсификацию,В условиях макроэкономической нестабильности и...
2,https://www.vedomosti.ru/ecology/climate/artic...,2022-12-12,"Метан – недолговечный, но коварный",Российские нефтегазовые компании активно участ...
3,https://www.vedomosti.ru/ecology/science_and_t...,2022-12-08,«Россия» станет самым мощным ледоколом в мире,"И. о. генерального директора ФГУП «Атомфлот», ..."
4,https://www.vedomosti.ru/ecology/science_and_t...,2022-12-08,Альтернатива «большому» атому,Их работа не ограничивается строительством и э...


In [15]:
df.columns = ['ссылка', 'дата', 'заголовок', 'текст']

In [16]:
df.head()

,ссылка,дата,заголовок,текст
0,https://www.vedomosti.ru/ecology/climate/colum...,2022-12-12,Климатические изменения и устойчивое развитие,Современные климатические изменения представля...
1,https://www.vedomosti.ru/ecology/science_and_t...,2022-12-12,Добывающей отрасли прогнозируют диверсификацию,В условиях макроэкономической нестабильности и...
2,https://www.vedomosti.ru/ecology/climate/artic...,2022-12-12,"Метан – недолговечный, но коварный",Российские нефтегазовые компании активно участ...
3,https://www.vedomosti.ru/ecology/science_and_t...,2022-12-08,«Россия» станет самым мощным ледоколом в мире,"И. о. генерального директора ФГУП «Атомфлот», ..."
4,https://www.vedomosti.ru/ecology/science_and_t...,2022-12-08,Альтернатива «большому» атому,Их работа не ограничивается строительством и э...


In [18]:
df.to_csv('Ecology.csv', encoding='utf-8') 

In [20]:
df.to_excel('Ecology.xlsx', encoding='utf-8' )